In [104]:
import bqplot.pyplot as plt
import bqplot as bq
import numpy as np

from IPython.display import display
from ipywidgets import widgets

In [105]:
class ZerosVisualizer:
    def __init__(self, fun, interval):
        self.x0, self.x1 = interval
        self.fun = fun
        self.xn = self.x0

        self.f0 = self.fun(self.x0)
        self.f1 = self.fun(self.x1)
        if self.f0 * self.f1 > 0:
            raise ValueError("The function has no zeros in the given interval")

        self.fn = self.f0

        self.e = self.x1 - self.xn

        self.iterations = 0

        self.x = np.linspace(self.x0, self.x1, 100)
        self.y = self.fun(self.x)

    def fixPoints(self):
        # Corrrects the point for the Algorithm to work
        if np.sign(self.f0) == np.sign(self.f1):
            self.x0 = self.xn
            self.f0 = self.fn
            self.e = self.x1 - self.xn
        elif np.sign(self.f0) < np.sign(self.f1):
            self.x0, self.x1, self.xn = self.x1, self.x0, self.x1
            self.f0, self.f1, self.fn = self.f1, self.f0, self.f1

    def newPoints(self):
        self.fixPoints()

        m = 0.5 * (self.x0 - self.x1)
        self.tol = 2.0 * np.finfo(float).eps * max(abs(self.x1), 1.0)
        if np.isclose(self.f1, 0) or abs(m) < self.tol:
            return "STOP"

        # Bisection
        self.bisectedStep = (self.x0, self.x1 + m, self.x1)
        # IQI Step
        s = self.f1 / self.fn
        if self.x0 == self.xn:
            # Linear interpolation
            p = 0
            q = 1.0 - s
        else:
            # Inverse quadratic interpolation
            q = self.fn / self.f0
            r = self.f1 / self.f0
            p = s * (2.0 * m * q * (q - r) - (self.x1 - self.xn) * (r - 1.0))
            q = (q - 1.0) * (r - 1.0) * (s - 1.0)

        if p > 0.0:
            q = -q
        else:
            p = -p

        d = p / q

        self.iqiStep = (self.x0, self.x1 + d, self.x1)
        if abs(d) < self.tol:
            self.iqiStep = (
                self.x0,
                self.x1 + np.sign(self.x1 - self.x0) * self.tol,
                self.x1,
            )

    def initializeComponents(self):

        self.revertButton = widgets.Button(description="Revert")
        # self.revertButton.on_click(self.revert)

        self.currentSolOut = widgets.Output()  # Current Solution: <x1>
        with self.currentSolOut:
            print("Current Solution: ", self.x1)

        self.helperOut = widgets.Output()  # Next Possible Step: <Bisect/IQI/None>

        # Reset Button
        self.resetButton = widgets.Button(
            description="Reset",
            disabled=False,
            button_style="danger",
            tooltip="Reset",
            icon="undo",
        )
        # self.resetButton.on_click(self.reset)

    def funPlot(self, x, y):
        self.funLine = bq.Lines(
            x=x,
            y=y,
            scales={"x": self.x_sc, "y": self.y_sc},
            colors="#808080",
            name="Real Function",
            display_legend=True,
            labels=["Real Function"],
            enable_move=False,
            enable_add=False,
            line_style="dashed",
        )

    def pointPlot(self, color, marker):
        # Scatter x0,x1 - f0,f1
        self.ogPoints = bq.Scatter(
            x=[self.x0, self.x1],
            y=[self.f0, self.f1],
            scales={"x": self.x_sc, "y": self.y_sc},
            colors=[color],
            name="Original Points",
            display_legend=False,
            labels=["Original Points"],
            enable_move=False,
            enable_add=False,
            marker=marker,
        )

        bisected = self.bisectedStep[1]
        self.bisectScatter = bq.Scatter(
            x=[bisected],
            y=[0],
            scales={"x": self.x_sc, "y": self.y_sc},
            colors=["blue"],
            name="Bisected Point",
            display_legend=True,
            labels=["Bisected Point"],
            enable_move=False,
            enable_add=False,
            marker="diamond",
        )
        self.bisectScatter.on_element_click(self.bisectClick)

        iqi = self.iqiStep[1]
        self.iqiScatter = bq.Scatter(
            x=[iqi],
            y=[0],
            scales={"x": self.x_sc, "y": self.y_sc},
            colors=["green"],
            name="IQI Point",
            display_legend=True,
            labels=["IQI Point"],
            enable_move=False,
            enable_add=False,
            marker="rectangle",
        )
        self.iqiScatter.on_element_click(self.iqiClick)

    def bisectClick(self, *args):
        self.x0, self.x1, self.xn = self.bisectedStep
        self.fixPoints()
        self.updateValues()

    def iqiClick(self, *args):
        self.e = -self.x1
        self.x0, self.x1, self.xn = self.bisectedStep
        self.e += self.x1
        self.fixPoints()
        self.updateValues()

    def updateValues(self):
        self.f0 = self.fun(self.x0)
        self.f1 = self.fun(self.x1)

        self.fn = self.fun(self.xn)

        self.iterations += 1

        print(self.f1)
        if (
            abs(self.f1) < 10**-2
            or abs(self.e) < self.tol
            or self.newPoints() == "STOP"
        ):
            self.helperOut.clear_output()
            with self.helperOut:
                print("Next Possible Step: None")
            self.bisectScatter.on_element_click(lambda *args: None)
            self.iqiScatter.on_element_click(lambda *args: None)
            return None

        self.pointPlot("Black", "cross")  # Plot the points

        if (self.x1 - self.x0) < 1:
            self.x_sc.min = min(self.x0, self.x1)
            self.x_sc.max = max(self.x0, self.x1)
            self.y_sc.min = min(self.f0, self.f1)
            self.y_sc.max = max(self.f0, self.f1)

            self.funPlot(
                np.linspace(self.x0, self.x1, 100),
                self.fun(np.linspace(self.x0, self.x1, 100)),
            )  # Plot the function

        self.Fig.marks = [
            self.Fig.marks[0],
            self.funLine,
            self.Fig.marks[2],
            self.ogPoints,
            self.bisectScatter,
            self.iqiScatter,
        ]

        with self.currentSolOut:
            print("Current Solution: ", self.x1)

    def runner(self):

        self.x_sc = bq.LinearScale()
        self.y_sc = bq.LinearScale()
        ax_x = bq.Axis(scale=self.x_sc, grid_lines="solid", label="X")
        ax_y = bq.Axis(
            scale=self.y_sc,
            orientation="vertical",
            tick_format="0.2f",
            grid_lines="solid",
            label="Y",
        )
        self.fixPoints()  # Corrects the points for the Algorithm to work
        self.initializeComponents()

        # Plots the function
        self.newPoints()  # Calculate the next points

        with self.helperOut:
            print(
                "Next Possible Step: ",
                "Bisect"
                if abs(self.e) < self.tol or abs(self.fn) < abs(self.f1)
                else "IQI",
            )

        self.pointPlot("Red", "circle")  # Plot the points
        self.funPlot(self.x, self.y)  # Plot the function
        horiLine = bq.Lines(
            x=[self.x0, self.x1],
            y=[0, 0],
            scales={"x": self.x_sc, "y": self.y_sc},
            colors="DimGray",
            name="Horizontal Line",
            display_legend=False,
            labels=["Horizontal Line"],
            enable_move=False,
            enable_add=False,
        )

        self.Fig = bq.Figure(
            marks=[
                horiLine,
                self.funLine,
                self.ogPoints,
                self.ogPoints,
                self.bisectScatter,
                self.iqiScatter,
            ],
            axes=[ax_x, ax_y],
            title="Zeros of a Function",
            # legend_location="top-right",
            animation_duration=1000,
        )
        display(self.Fig)  # Display the figure

In [106]:
f = lambda x: (x - 1) * (x - 4) * (x - 3)
interval = (0, 5)

zv = ZerosVisualizer(f, interval)

zv.runner()  # Run the visualizer

Figure(animation_duration=1000, axes=[Axis(label='X', scale=LinearScale()), Axis(label='Y', orientation='verti…

1.125
-12
-3.005859375
-0.394775390625
0.519378662109375
-0.394775390625
-0.14338445663452148
-0.023513853549957275
0.03498478978872299
-0.023513853549957275
-0.00879979447927326
-0.00879979447927326
-0.00879979447927326
-0.00879979447927326
-0.00879979447927326
